In [4]:
import torch
#import time
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim, autograd
from torch.nn import functional as F
#from torch.distributions.multivariate_normal import MultivariateNormal
import numpy as np
#from sklearn.utils import shuffle as skshuffle
import math
from backpack import backpack, extend
from backpack.extensions import KFAC, DiagHessian
#from sklearn.metrics import roc_auc_score
#import scipy
#from tqdm import tqdm, trange
#import pytest
#import matplotlib.pyplot as plt
#from DirLPA_utils import *
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

print("pytorch version: ", torch.__version__)
print("cuda available: ", torch.cuda.is_available())

s = 20
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



pytorch version:  1.5.0
cuda available:  False


In [2]:
#DATA WRANGLING

# load SVHN
BATCH_SIZE_TRAIN_SVHN = 128
BATCH_SIZE_TEST_SVHN = 128

transform_base = [transforms.ToTensor()]

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    ] + transform_base)

transform_test = transforms.Compose(transform_base)
transform_train = transforms.RandomChoice([transform_train, transform_test])

#'~/data/SVHN'
train_data_SVHN = torchvision.datasets.SVHN('~/data/SVHN', split='train',
                             download=True, transform=transform_train)

#'~/data/SVHN'
test_data_SVHN = torchvision.datasets.SVHN('~/data/SVHN', split='test',
                             download=True, transform=transform_test)

SVHN_train_loader = torch.utils.data.DataLoader(train_data_SVHN, batch_size=BATCH_SIZE_TRAIN_SVHN)
SVHN_test_loader = torch.utils.data.DataLoader(test_data_SVHN, batch_size=BATCH_SIZE_TEST_SVHN)




Using downloaded and verified file: /Users/moreez/data/SVHN/train_32x32.mat
Using downloaded and verified file: /Users/moreez/data/SVHN/test_32x32.mat


In [7]:

sz=16

def vggNet2():
    layers = nn.Sequential()
    layers.add_module('0', nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('1', nn.ReLU())
    layers.add_module('2', nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
    layers.add_module('3', nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('4', nn.ReLU())
    layers.add_module('5', nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
    layers.add_module('6', nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('7', nn.ReLU())
    layers.add_module('8', nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('9', nn.ReLU())
    layers.add_module('10', nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
    layers.add_module('11', nn.Conv2d(256, sz, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('12', nn.ReLU())
    layers.add_module('13', nn.Conv2d(sz, sz, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('14', nn.ReLU())
    layers.add_module('15', nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
    layers.add_module('16', nn.Conv2d(sz, sz, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('17', nn.ReLU())
    layers.add_module('18', nn.Conv2d(sz, sz, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
    layers.add_module('19', nn.ReLU())
    layers.add_module('20', nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
    # layers.add_module('21', nn.AdaptiveAvgPool2d(output_size=(7, 7)))
    layers.add_module('21', nn.Flatten())
    layers.add_module('22', nn.Dropout(p=0.5, inplace=False))
    layers.add_module('23', nn.Linear(in_features=sz, out_features=sz, bias=True))
    layers.add_module('24', nn.ReLU())
    layers.add_module('25', nn.Dropout(p=0.5, inplace=False))
    layers.add_module('26', nn.Linear(in_features=sz, out_features=sz, bias=True))
    layers.add_module('27', nn.ReLU())
    layers.add_module('28', nn.Linear(in_features=sz, out_features=10, bias=True))
    return layers


def init_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
            m.bias.data.zero_()

#SVHN_model = vggNet2()
#init_weights(SVHN_model)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
cuda_status = torch.cuda.is_available()
SVHN_model = vggNet2().to(device)
init_weights(SVHN_model)
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(SVHN_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)


def train(net, epoch, optimizer, trainloader, filename):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print("train loss: ", train_loss)
    print("train accuracy: ", correct / total)
    print("saving model at: {}".format(filename))
    torch.save(net.state_dict(), filename)


def test(net, epoch, testloader, path, save=False):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        acc = correct / total
        if acc > best_acc and save:
            best_acc = acc
            print("saving model at: {}".format(path))
            torch.save(net.state_dict(), path)

        print("test loss: ", test_loss)
        print("current acc: {}; best acc: {}".format(acc, best_acc))

def train_all():
    SVHN_path = './trained_weights/{}ckpt_seed{}.pth'.format(sz, s)
    SVHN_path_best = './trained_weights/{}best_ckpt_seed{}.pth'.format(sz, s)
    lr = 0.1
    epoch = 0
    for e in [30, 50, 50]:
        print("current learning rate: ", lr)
        for _ in range(e):
            optimizer = optim.SGD(SVHN_model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
            train(SVHN_model, epoch, optimizer, SVHN_train_loader, SVHN_path)
            test(SVHN_model, epoch, SVHN_test_loader, save=True, path=SVHN_path_best)
            epoch += 1
        lr /= 10

train_all()

print('trained and saved the network')




current learning rate:  0.1

Epoch: 0
train loss:  1283.7297031879425
train accuracy:  0.18772267496621484
saving model at: ./trained_weights/16ckpt_seed20.pth
saving model at: ./trained_weights/16best_ckpt_seed20.pth
test loss:  454.7961654663086
current acc: 0.1958743085433313; best acc: 0.1958743085433313

Epoch: 1
train loss:  1282.6570055484772
train accuracy:  0.18921058738414076
saving model at: ./trained_weights/16ckpt_seed20.pth
test loss:  454.75408124923706
current acc: 0.1958743085433313; best acc: 0.1958743085433313

Epoch: 2
train loss:  1282.648553609848
train accuracy:  0.18921058738414076
saving model at: ./trained_weights/16ckpt_seed20.pth
test loss:  454.75234508514404
current acc: 0.1958743085433313; best acc: 0.1958743085433313

Epoch: 3
train loss:  1282.6471180915833
train accuracy:  0.18921058738414076
saving model at: ./trained_weights/16ckpt_seed20.pth
test loss:  454.7524268627167
current acc: 0.1958743085433313; best acc: 0.1958743085433313

Epoch: 4
train l

KeyboardInterrupt: 